Reference script directory:

- work/text_analysis/parsing/Overlap_Parsing_Improved.py : removed overlaps in the webtext
- work/text_analysis/parsing/filter_pages.py : gets the top 250 most relevan
t pages
- work/text_analysis/parsing/further_cleaning_script.ipynb : string cleaning methods
    

Use file /vol_b/data/misc_data/webtext_raw/webtext_unlapped_full.tsv as data source. We are most interested in the "text_full" column: this is a list of quadruples (one per web page), where 4th element is the text for a web page.

Filter each website to its 250 most important pages--that is, keep the top 250 ranked pages and no more. Rank pages based on their number of occurrences of the following keywords:

keywords = ['values', 'academics', 'academic', 'skills', 'skill', 'purpose', 'purposes', 'direction', 'mission', 'vision', 'visions', 'missions', 'ideals', 'cause', 'causes', 'curriculum', 'curricular', 'method', 'methods', 'pedagogy', 'pedagogical', 'pedagogies', 'approach', 'approaches', 'model', 'models', 'system', 'systems', 'structure', 'structures', 'philosophy', 'philosophical', 'philosophies', 'beliefs', 'believe', 'belief', 'principles', 'principle', 'creed', 'creeds', 'credo', 'moral', 'morals', 'morality', 'history', 'histories', 'our story', 'the story', 'school story', 'background', 'backgrounds', 'founding', 'founded', 'foundation', 'foundations', 'foundational', 'established','establishment', 'our school began', 'we began', 'doors opened', 'school opened', 'about us', 'our school', 'who we are', 'identity', 'identities', 'profile', 'highlights']

Save the resulting file as /vol_b/data/misc_data/webtext_raw/webtext_unlapped_filtered_250.tsv.

In [26]:
import pandas as pd

In [32]:
original = pd.read_csv("/home/jovyan/work/misc_data/webtext_raw/webtext_unlapped_full.tsv", sep="\t")

In [33]:
original = original.drop(['Unnamed: 0'], axis=1)

convert str to lst

In [34]:
original["text_full"] = original["text_full"].apply(lambda x:eval(x))

sanity check df

In [35]:
original.head()

,text_full,NCESSCH
0,"[(http://www.maef.net/, False, 0, Evening Acad...",1.001970e+10
1,"[(https://www.kgbsd.org/ketchikancharter, Fals...",2.001500e+10
2,[(http://tongassschool.org/classrooms/3rd-and-...,2.001500e+10
3,"[(https://www.asdk12.org/aquarian, False, 0, S...",2.001800e+10
4,"[(http://winterberrycharterschool.com/, False,...",2.001800e+10


In [56]:
original.shape

(6462, 3)

def keywords

In [63]:
keywords = ['values', 'academics', 'academic', 'skills', 'skill', 'purpose', 'purposes', 'direction', 'mission', 'vision', 'visions', 'missions', 'ideals', 'cause', 'causes', 'curriculum', 'curricular', 'method', 'methods', 'pedagogy', 'pedagogical', 'pedagogies', 'approach', 'approaches', 'model', 'models', 'system', 'systems', 'structure', 'structures', 'philosophy', 'philosophical', 'philosophies', 'beliefs', 'believe', 'belief', 'principles', 'principle', 'creed', 'creeds', 'credo', 'moral', 'morals', 'morality', 'history', 'histories', 'our story', 'the story', 'school story', 'background', 'backgrounds', 'founding', 'founded', 'foundation', 'foundations', 'foundational', 'established','establishment', 'our school began', 'we began', 'doors opened', 'school opened', 'about us', 'our school', 'who we are', 'identity', 'identities', 'profile', 'highlights']

count page num for each row (before cleaning)

In [52]:
original["page_num"] = original["text_full"].apply(lambda x:len(x))

sanity check rows which have more than 250 pages

In [54]:
original[original['page_num'] > 250] 

,text_full,NCESSCH,page_num
48,"[(https://www.goasa.org/, False, 0, 2.5 hours ...",4.000560e+10,270
108,"[(https://www.alaschools.org/, False, 0, Blog\...",4.001120e+10,484
109,"[(https://www.alaschools.org/, False, 0, Blog\...",4.001120e+10,487
112,"[(https://www.alaschools.org/, False, 0, Blog\...",4.001120e+10,494
121,"[(https://www.allaccelerated.org/tucson/, Fals...",4.001290e+10,270
...,...,...,...
6384,"[(https://www.newrichmond.k12.wi.us/, False, 0...",5.510620e+11,658
6406,"[(http://www.rbsd.k12.wi.us/, False, 0, r\nIt ...",5.513100e+11,386
6420,"[(http://www.spartan.org/, False, 0, 201 E. Fr...",5.514160e+11,1067
6424,"[(http://www.spooner.k12.wi.us/, False, 0, Sea...",5.514220e+11,300


count keywords in each page, sort by count

In [143]:
def get250(lst):
    if len(lst) <= 250:
        return lst
    else:
        dic = {}
        for page in lst:
            count = 0
            for w in page[3].split():
#                 print(w)
                if w.lower() in keywords:
                    count += 1
            dic[page] = count
        return sorted(dic, key = dic.get, reverse=True)[:250]

apply function to each row

In [144]:
original['text_cleaned'] = original['text_full'].apply(lambda x: get250(x))

check result

In [154]:
original.head()

,text_full,NCESSCH,page_num,text_cleaned
0,"[(http://www.maef.net/, False, 0, Evening Acad...",1.001970e+10,41,"[(http://www.maef.net/, False, 0, Evening Acad..."
1,"[(https://www.kgbsd.org/ketchikancharter, Fals...",2.001500e+10,1,"[(https://www.kgbsd.org/ketchikancharter, Fals..."
2,[(http://tongassschool.org/classrooms/3rd-and-...,2.001500e+10,117,[(http://tongassschool.org/classrooms/3rd-and-...
3,"[(https://www.asdk12.org/aquarian, False, 0, S...",2.001800e+10,1,"[(https://www.asdk12.org/aquarian, False, 0, S..."
4,"[(http://winterberrycharterschool.com/, False,...",2.001800e+10,33,"[(http://winterberrycharterschool.com/, False,..."


save as csv

In [159]:
original.to_csv("/home/jovyan/work/misc_data/webtext_raw/webtext_cleaned_250.csv")

check if successfully saved

In [160]:
cleaned = pd.read_csv("/home/jovyan/work/misc_data/webtext_raw/webtext_cleaned_250.csv")

In [161]:
cleaned.head()

,Unnamed: 0,text_full,NCESSCH,page_num,text_cleaned
0,0,"[('http://www.maef.net/', 'False', '0', ""Eveni...",1.001970e+10,41,"[('http://www.maef.net/', 'False', '0', ""Eveni..."
1,1,"[('https://www.kgbsd.org/ketchikancharter', 'F...",2.001500e+10,1,"[('https://www.kgbsd.org/ketchikancharter', 'F..."
2,2,[('http://tongassschool.org/classrooms/3rd-and...,2.001500e+10,117,[('http://tongassschool.org/classrooms/3rd-and...
3,3,"[('https://www.asdk12.org/aquarian', 'False', ...",2.001800e+10,1,"[('https://www.asdk12.org/aquarian', 'False', ..."
4,4,"[('http://winterberrycharterschool.com/', 'Fal...",2.001800e+10,33,"[('http://winterberrycharterschool.com/', 'Fal..."
